In [1]:
# Dependencies and Setup
import pandas as pd

# File to Load (Remember to Change These)
file_to_load = "Resources/purchase_data.csv"

# Read Purchasing File and store into Pandas data frame
purchase_data = pd.read_csv(file_to_load)
purchase_data.head()

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,2,Ithergue48,24,Male,92,Final Critic,4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27
4,4,Iskosia90,23,Male,131,Fury,1.44


In [2]:
# calculate player count
total_players =purchase_data["SN"].nunique()
# purchase_data["SN"].nunique()
print(f"Player count is {total_players}")


Player count is 576


In [3]:
# calculate metrics for purchasing analysis summary
unique_items = len(purchase_data["Item ID"].unique())
avg_price = purchase_data["Price"].mean()
num_purchases = purchase_data["SN"].count()
tot_rev = purchase_data["Price"].sum()
# print(unique_items)
# print(avg_price)
# print(num_purchases)
# print(tot_rev)

In [4]:
# Place all calculated data in a summary DF
summary_df = pd.DataFrame({"Number of Unique Items": [unique_items], 
                           "Average Price": avg_price, 
                           "Number of Purchases": num_purchases,"Total Revenue": tot_rev})
summary_df["Average Price"] = summary_df["Average Price"].map("${0: .2f}".format)
summary_df["Total Revenue"] = summary_df["Total Revenue"].map("${0: .2f}".format)

summary_df

,Number of Unique Items,Average Price,Number of Purchases,Total Revenue
0,179,$ 3.05,780,$ 2379.77


In [5]:
# calculate percentage and count of gender
gender = purchase_data.groupby("Gender")["SN"].nunique()
gender = gender.to_frame()

percent_gender = gender["SN"]/total_players
gender["Percentage of Players"] = percent_gender
gender = gender.rename(columns={"SN":"Total Count"})

female_total = gender.iloc[0,0]
male_total = gender.iloc[1,0]
other_total = gender.iloc[2,0]

gender["Percentage of Players"] = gender["Percentage of Players"].map("{: .2%}".format)

gender


,Total Count,Percentage of Players
Gender,,
Female,81,14.06%
Male,484,84.03%
Other / Non-Disclosed,11,1.91%


In [6]:
# calculate metrics for gender purchase analysis
purchase_count_df = purchase_data.groupby("Gender")["SN"].count()
avg_price_df = purchase_data.groupby("Gender")["Price"].mean()
total_value_df = purchase_data.groupby("Gender")["Price"].sum()


# merge dataframes into one
merge1_df = pd.merge(purchase_count_df,avg_price_df, on="Gender")
merge2_df = pd.merge(merge1_df,total_value_df, on="Gender")
merge3_df = pd.merge(merge2_df, gender, on="Gender")


In [7]:
# Calculate avg purchase per person and add to dataframe
avg_purchase = (merge3_df["Price_y"] / merge3_df["Total Count"])
merge3_df["Avg_pur_per_person"] = avg_purchase
# merge3_df

In [22]:
# final gender purchase analysis dataframe with formatting
final_merge = merge3_df.rename(columns={"SN": "Purchase Count","Price_x": "Average Purchase Price","Price_y": "Total Purchase Price","SN_y": "Total Gender Count","Avg_pur_per_person": "Avg Total Purchase per Person"})
final_merge = final_merge.drop(final_merge.columns[[3,4]], axis=1)
format_dict = {"Average Purchase Price": "${0:,.2f}", "Total Purchase Price": "${0:,.2f}","Avg Total Purchase per Person": "${0:,.2f}","Percentage of Players": "{:.2%}"}
final_merge.style.format(format_dict)

,Purchase Count,Average Purchase Price,Total Purchase Price,Avg Total Purchase per Person
Gender,,,,
Female,113,$3.20,$361.94,$4.47
Male,652,$3.02,"$1,967.64",$4.07
Other / Non-Disclosed,15,$3.35,$50.19,$4.56


In [9]:
# Create the bins in which Data will be held
# Bins are 0, 10, 15, 20, 25, 30, 35, 40, 100   
bins = [0, 9, 14, 19, 24, 29, 34, 39, 100]

# Create the names for the five bins
group_names = ["<10", "10-14", "15-19", "20-24", "25-29", "30-34", "35-39", "40+"]

In [10]:
# create new dataframe with binned data
purchase_data["Age Bin"] = pd.cut(purchase_data["Age"], bins, labels=group_names, include_lowest=True)
# purchase_data

In [24]:
# calculate purchase analysis by bins
age_df = purchase_data.groupby("Age Bin")
bin_df = age_df.nunique()
perc_player = (bin_df["SN"] / 576)
bin_df["Percentage of Players"] = perc_player
# format_dict = {"Percentage of Players": "{:.2%}"}
# bin_df.style.format(format_dict)

In [12]:
# final dataframe with data by bins
final_bin = bin_df.drop(bin_df.columns[[0,2,3,4,5,6,7]], axis=1)

final_bin = final_bin.rename(columns={"SN": "Total Count"})
format_dict = {'Percentage of Players': '{:.2%}'}
final_bin.style.format(format_dict)
final_bin["Percentage of Players"] = final_bin["Percentage of Players"].map("{: .2%}".format)
final_bin

,Total Count,Percentage of Players
Age Bin,,
<10,17,2.95%
10-14,22,3.82%
15-19,107,18.58%
20-24,258,44.79%
25-29,77,13.37%
30-34,52,9.03%
35-39,31,5.38%
40+,12,2.08%


In [13]:
# calculate data for purchasing analysis by age
purchase_count_df = purchase_data.groupby("Age Bin")["Price"].count()
avg_price_df = purchase_data.groupby("Age Bin")["Price"].mean()
total_value_df = purchase_data.groupby("Age Bin")["Price"].sum()
# purchase_count_df
# avg_price_df
# total_value_df
merge4_df = pd.merge(purchase_count_df,avg_price_df, on="Age Bin")
merge5_df = pd.merge(merge4_df,total_value_df, on="Age Bin")
avg_total_pp = merge5_df["Price"] / bin_df["SN"]
merge5_df["Avg_pur_per_person"] = avg_total_pp
# merge5_df

In [14]:
# final merge of data
purchase_analysis_df = merge5_df.rename(columns={"Price_x": "Purchase Count","Price_y": "Average Purchase Price","Price": "Total Purchase Price","Avg_pur_per_person": "Avg Total Purchase per Person"})

# purchase_analysis_df

In [15]:
# print dataframe with formatting
purchase_analysis_df["Average Purchase Price"] = purchase_analysis_df["Average Purchase Price"].map("${0:,.2f}".format)
purchase_analysis_df["Total Purchase Price"] = purchase_analysis_df["Total Purchase Price"].map("${0:,.2f}".format)
purchase_analysis_df["Avg Total Purchase per Person"] = purchase_analysis_df["Avg Total Purchase per Person"].map("${0:,.2f}".format)

purchase_analysis_df

,Purchase Count,Average Purchase Price,Total Purchase Price,Avg Total Purchase per Person
Age Bin,,,,
<10,23,$3.35,$77.13,$4.54
10-14,28,$2.96,$82.78,$3.76
15-19,136,$3.04,$412.89,$3.86
20-24,365,$3.05,"$1,114.06",$4.32
25-29,101,$2.90,$293.00,$3.81
30-34,73,$2.93,$214.00,$4.12
35-39,41,$3.60,$147.67,$4.76
40+,13,$2.94,$38.24,$3.19


In [16]:
# calculate data for top spenders analysis
purchasect = purchase_data.groupby("SN")["Purchase ID"].nunique()
avgpurchase = purchase_data.groupby("SN")["Price"].mean()
totalvalue = purchase_data.groupby("SN")["Price"].sum()

top_spenders_df = pd.DataFrame({"Purchase Count": purchasect, "Average Purchase Price": avgpurchase, "Total Purchase Value": totalvalue})


# top_spenders_df

In [17]:
# Sort based on total value
sorted_df = top_spenders_df.sort_values("Total Purchase Value", ascending=False)
sorted_df["Total Purchase Value"] = sorted_df["Total Purchase Value"].map("${0:,.2f}".format)
sorted_df["Average Purchase Price"] = sorted_df["Average Purchase Price"].map("${0:,.2f}".format)

sorted_df.head(5)

,Purchase Count,Average Purchase Price,Total Purchase Value
SN,,,
Lisosia93,5,$3.79,$18.96
Idastidru52,4,$3.86,$15.45
Chamjask73,3,$4.61,$13.83
Iral74,4,$3.40,$13.62
Iskadarya95,3,$4.37,$13.10


In [18]:
# extract 3 columns from original DF and create new DF
popular_df = purchase_data[["Item ID","Item Name","Price"]]
# popular_df

In [19]:
# group by item id and name and create new DF
grouped_items  = popular_df.groupby(['Item ID', 'Item Name'])
# grouped_items  = popular_df.groupby('Item ID')['Item Name']

# grouped_items["Price"].mean()
df2 = pd.DataFrame(grouped_items["Price"].mean())
# df2

In [20]:
# calculated data for analysis and created DF, sorted and formatted
mostpop_value = pd.DataFrame(grouped_items["Price"].sum())
mostpop_count = pd.DataFrame(grouped_items["Item Name"].count())
# mostpop_price = pd.DataFrame(grouped_items["Price"].mean())
# mostpop_price

df2["Purchase Count"] = mostpop_count
df2["Total Purchase Value"] = mostpop_value
df3 = df2[["Purchase Count","Price","Total Purchase Value"]]
sorted_popular_df = df3.sort_values("Purchase Count", ascending=False)
sorted_popular_df["Price"] = sorted_popular_df["Price"].map("${0:,.2f}".format)
sorted_popular_df["Total Purchase Value"] = sorted_popular_df["Total Purchase Value"].map("${0:,.2f}".format)

sorted_popular_df.head(10)

,,Purchase Count,Price,Total Purchase Value
Item ID,Item Name,,,
92,Final Critic,13,$4.61,$59.99
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
145,Fiery Glass Crusader,9,$4.58,$41.22
132,Persuasion,9,$3.22,$28.99
108,"Extraction, Quickblade Of Trembling Hands",9,$3.53,$31.77
82,Nirvana,9,$4.90,$44.10
75,Brutality Ivory Warmace,8,$2.42,$19.36
103,Singed Scalpel,8,$4.35,$34.80
34,Retribution Axe,8,$2.22,$17.76


In [21]:
# sorted same DF for most profitable 
sorted_popular_df = df3.sort_values("Total Purchase Value", ascending=False)
sorted_popular_df["Price"] = sorted_popular_df["Price"].map("${0:,.2f}".format)
sorted_popular_df["Total Purchase Value"] = sorted_popular_df["Total Purchase Value"].map("${0:,.2f}".format)

sorted_popular_df.head()

,,Purchase Count,Price,Total Purchase Value
Item ID,Item Name,,,
92,Final Critic,13,$4.61,$59.99
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
82,Nirvana,9,$4.90,$44.10
145,Fiery Glass Crusader,9,$4.58,$41.22
103,Singed Scalpel,8,$4.35,$34.80
